In [1]:
import numpy as np
import warnings 
import sys 
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/enb.arff')
data = pd.DataFrame(data)
data.head()

,Relative_compactness,X1,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,0.0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2.0,0.0,0.0,20.84,28.28


In [3]:
X = data.iloc[:,:-2].values
y = data.iloc[:,-2:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(768, 8) (768, 2)


In [4]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(537, 8) (537, 2) (231, 8) (231, 2)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 768
    p = 8

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [8]:
ENS_COUNT = 4

In [9]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    
    return model

### Forward Ensemble

In [10]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_forward = []
    
    X_feed = X_train.copy()
    y_t = y_train.copy()
    
    for i in range(y_t.shape[1]):
        print('Training chain node ',i)
        y_now = y_t[:,[i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_forward.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_fwd.append(chain_forward)

Training Ensemble 0 - 

Training chain node  0
Shapes - 
 X = (537, 8) 
 Y = (537, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
537/537 [==============================] - 0s - loss: 0.4328 - mean_squared_error: 0.4328     
Epoch 2/500
537/537 [==============================] - 0s - loss: 0.1444 - mean_squared_error: 0.1444     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.1010 - mean_squared_error: 0.1010     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.0863 - mean_squared_error: 0.0863     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.0723 - mean_squared_error: 0.0723     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.0668 - mean_squared_error: 0.0668     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0612 - mean_squared_error: 0.0612     
Epoch 8/500
537/537 [====================

537/537 [==============================] - 0s - loss: 0.0048 - mean_squared_error: 0.0048     
Epoch 63/500
537/537 [==============================] - 0s - loss: 0.0049 - mean_squared_error: 0.0049     
Epoch 64/500
537/537 [==============================] - 0s - loss: 0.0048 - mean_squared_error: 0.0048     
Epoch 65/500
537/537 [==============================] - ETA: 0s - loss: 0.0045 - mean_squared_error: 0.00 - 0s - loss: 0.0043 - mean_squared_error: 0.0043     
Epoch 66/500
537/537 [==============================] - 0s - loss: 0.0040 - mean_squared_error: 0.0040     
Epoch 67/500
537/537 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 68/500
537/537 [==============================] - 0s - loss: 0.0035 - mean_squared_error: 0.0035     
Epoch 69/500
537/537 [==============================] - 0s - loss: 0.0041 - mean_squared_error: 0.0041     
Epoch 70/500
537/537 [==============================] - 0s - loss: 0.0040 - mean_squared_error: 0

537/537 [==============================] - 0s - loss: 8.0822e-04 - mean_squared_error: 8.0822e-04     
Epoch 136/500
537/537 [==============================] - 0s - loss: 8.7191e-04 - mean_squared_error: 8.7191e-04     
Epoch 137/500
537/537 [==============================] - 0s - loss: 8.3709e-04 - mean_squared_error: 8.3709e-04
Epoch 138/500
537/537 [==============================] - 0s - loss: 7.4910e-04 - mean_squared_error: 7.4910e-04     
Epoch 139/500
537/537 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012             
Epoch 140/500
537/537 [==============================] - 0s - loss: 8.9555e-04 - mean_squared_error: 8.9555e-04     
Epoch 141/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 142/500
537/537 [==============================] - 0s - loss: 9.1155e-04 - mean_squared_error: 9.1155e-04
Epoch 143/500
537/537 [==============================] - 0s - loss: 0.0010 - mean_squared_error:

537/537 [==============================] - 0s - loss: 5.7595e-04 - mean_squared_error: 5.7595e-04     
Epoch 206/500
537/537 [==============================] - 0s - loss: 6.1669e-04 - mean_squared_error: 6.1669e-04     
Epoch 207/500
537/537 [==============================] - 0s - loss: 6.2413e-04 - mean_squared_error: 6.2413e-04     
Epoch 208/500
537/537 [==============================] - 0s - loss: 4.6225e-04 - mean_squared_error: 4.6225e-04     
Epoch 209/500
537/537 [==============================] - 0s - loss: 5.7542e-04 - mean_squared_error: 5.7542e-04     
Epoch 210/500
537/537 [==============================] - 0s - loss: 7.0452e-04 - mean_squared_error: 7.0452e-04     
Epoch 211/500
537/537 [==============================] - 0s - loss: 9.4239e-04 - mean_squared_error: 9.4239e-04     
Epoch 212/500
537/537 [==============================] - 0s - loss: 7.1893e-04 - mean_squared_error: 7.1893e-04     
Epoch 213/500
537/537 [==============================] - ETA: 0s - loss: 0.001

537/537 [==============================] - 0s - loss: 4.9772e-04 - mean_squared_error: 4.9772e-04     
Epoch 276/500
537/537 [==============================] - 0s - loss: 5.4400e-04 - mean_squared_error: 5.4400e-04     
Epoch 277/500
537/537 [==============================] - 0s - loss: 8.9772e-04 - mean_squared_error: 8.9772e-04     
Epoch 278/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013             
Epoch 279/500
537/537 [==============================] - 0s - loss: 7.8284e-04 - mean_squared_error: 7.8284e-04
Epoch 280/500
537/537 [==============================] - 0s - loss: 8.0748e-04 - mean_squared_error: 8.0748e-04     
Epoch 281/500
537/537 [==============================] - 0s - loss: 5.5717e-04 - mean_squared_error: 5.5717e-04     
Epoch 282/500
537/537 [==============================] - 0s - loss: 9.6299e-04 - mean_squared_error: 9.6299e-04     
Epoch 283/500
537/537 [==============================] - 0s - loss: 8.5533e-04 - me

...

537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 213/500
537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 214/500
537/537 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 215/500
537/537 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 216/500
537/537 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 217/500
537/537 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016             
Epoch 218/500
537/537 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 219/500
537/537 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 220/500
537/537 [==============================] - 0s - loss: 0.0028 - mean_squared_error: 0.0028     
Epoch 221/500
537/537 [==

537/537 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010     
Epoch 285/500
537/537 [==============================] - 0s - loss: 8.3057e-04 - mean_squared_error: 8.3057e-04     
Epoch 286/500
537/537 [==============================] - 0s - loss: 8.9965e-04 - mean_squared_error: 8.9965e-04
Epoch 287/500
537/537 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010     
Epoch 288/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 289/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 290/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 291/500
537/537 [==============================] - 0s - loss: 8.7907e-04 - mean_squared_error: 8.7907e-04     
Epoch 292/500
537/537 [==============================] - 0s - loss: 7.5446e-04 - mean_squared_error: 7.5446e-04     
Epoch 

537/537 [==============================] - 0s - loss: 5.7149e-04 - mean_squared_error: 5.7149e-04     
Epoch 355/500
537/537 [==============================] - 0s - loss: 6.7021e-04 - mean_squared_error: 6.7021e-04     
Epoch 356/500
537/537 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010             
Epoch 357/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 358/500
537/537 [==============================] - 0s - loss: 0.0038 - mean_squared_error: 0.0038     
Epoch 359/500
537/537 [==============================] - 0s - loss: 0.0028 - mean_squared_error: 0.0028     
Epoch 360/500
537/537 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 361/500
537/537 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 362/500
537/537 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 363

537/537 [==============================] - 0s - loss: 3.1430e-04 - mean_squared_error: 3.1430e-04     
Epoch 426/500
537/537 [==============================] - 0s - loss: 2.9006e-04 - mean_squared_error: 2.9006e-04     
Epoch 427/500
537/537 [==============================] - 0s - loss: 4.2238e-04 - mean_squared_error: 4.2238e-04     
Epoch 428/500
537/537 [==============================] - 0s - loss: 3.3943e-04 - mean_squared_error: 3.3943e-04     
Epoch 429/500
537/537 [==============================] - 0s - loss: 3.5863e-04 - mean_squared_error: 3.5863e-04     
Epoch 430/500
537/537 [==============================] - 0s - loss: 3.9754e-04 - mean_squared_error: 3.9754e-04     
Epoch 431/500
537/537 [==============================] - 0s - loss: 5.9044e-04 - mean_squared_error: 5.9044e-04     
Epoch 432/500
537/537 [==============================] - 0s - loss: 4.9855e-04 - mean_squared_error: 4.9855e-04     
Epoch 433/500
537/537 [==============================] - 0s - loss: 4.3836e-04

537/537 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 496/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 497/500
537/537 [==============================] - 0s - loss: 8.3510e-04 - mean_squared_error: 8.3510e-04     
Epoch 498/500
537/537 [==============================] - 0s - loss: 7.3473e-04 - mean_squared_error: 7.3473e-04     
Epoch 499/500
537/537 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014     
Epoch 500/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013             
Training of node 1 complete



### Backward Ensemble

In [11]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_reverse = []
    
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print('Training data - ',i)
        y_now = y_t[:,[-i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_reverse.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_rev.append(chain_reverse)

Training Ensemble 0 - 

Training data -  1
Shapes - 
 X = (537, 8) 
 Y = (537, 1)
Epoch 1/500
537/537 [==============================] - 0s - loss: 0.8902 - mean_squared_error: 0.8902     
Epoch 2/500
537/537 [==============================] - 0s - loss: 0.3402 - mean_squared_error: 0.3402     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.1804 - mean_squared_error: 0.1804     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.1318 - mean_squared_error: 0.1318     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.1144 - mean_squared_error: 0.1144     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.1058 - mean_squared_error: 0.1058     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0964 - mean_squared_error: 0.0964     
Epoch 8/500
537/537 [==============================] - 0s - loss: 0.0903 - mean_squared_error: 0.0903     
Epoch 9/500
537/537 [==============================] - 0s - lo

537/537 [==============================] - 0s - loss: 0.0076 - mean_squared_error: 0.0076     
Epoch 77/500
537/537 [==============================] - 0s - loss: 0.0081 - mean_squared_error: 0.0081     
Epoch 78/500
537/537 [==============================] - 0s - loss: 0.0075 - mean_squared_error: 0.0075     
Epoch 79/500
537/537 [==============================] - 0s - loss: 0.0075 - mean_squared_error: 0.0075     
Epoch 80/500
537/537 [==============================] - 0s - loss: 0.0080 - mean_squared_error: 0.0080     
Epoch 81/500
537/537 [==============================] - 0s - loss: 0.0079 - mean_squared_error: 0.0079     
Epoch 82/500
537/537 [==============================] - 0s - loss: 0.0071 - mean_squared_error: 0.0071     
Epoch 83/500
537/537 [==============================] - 0s - loss: 0.0071 - mean_squared_error: 0.0071     
Epoch 84/500
537/537 [==============================] - 0s - loss: 0.0067 - mean_squared_error: 0.0067     
Epoch 85/500
537/537 [===================

537/537 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032     
Epoch 152/500
537/537 [==============================] - 0s - loss: 0.0031 - mean_squared_error: 0.0031     
Epoch 153/500
537/537 [==============================] - 0s - loss: 0.0033 - mean_squared_error: 0.0033     
Epoch 154/500
537/537 [==============================] - 0s - loss: 0.0031 - mean_squared_error: 0.0031     
Epoch 155/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 156/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 157/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 158/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 159/500
537/537 [==============================] - 0s - loss: 0.0025 - mean_squared_error: 0.0025     
Epoch 160/500
537/537 [==========

537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 227/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 228/500
537/537 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014     
Epoch 229/500
537/537 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 230/500
537/537 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 231/500
537/537 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022     
Epoch 232/500
537/537 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 233/500
537/537 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 234/500
537/537 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022     
Epoch 235/500
537/537 [==========

537/537 [==============================] - 0s - loss: 6.9599e-04 - mean_squared_error: 6.9599e-04     
Epoch 300/500
537/537 [==============================] - 0s - loss: 7.5335e-04 - mean_squared_error: 7.5335e-04     
Epoch 301/500
537/537 [==============================] - 0s - loss: 5.1030e-04 - mean_squared_error: 5.1030e-04     
Epoch 302/500
537/537 [==============================] - 0s - loss: 7.1609e-04 - mean_squared_error: 7.1609e-04     
Epoch 303/500
537/537 [==============================] - 0s - loss: 5.9083e-04 - mean_squared_error: 5.9083e-04     
Epoch 304/500
537/537 [==============================] - 0s - loss: 6.1873e-04 - mean_squared_error: 6.1873e-04     
Epoch 305/500
537/537 [==============================] - 0s - loss: 4.7111e-04 - mean_squared_error: 4.7111e-04     
Epoch 306/500
537/537 [==============================] - 0s - loss: 4.7390e-04 - mean_squared_error: 4.7390e-04     
Epoch 307/500
537/537 [==============================] - 0s - loss: 6.9265e-04

...

537/537 [==============================] - 0s - loss: 7.9145e-04 - mean_squared_error: 7.9145e-04     
Epoch 157/500
537/537 [==============================] - 0s - loss: 5.5464e-04 - mean_squared_error: 5.5464e-04     
Epoch 158/500
537/537 [==============================] - 0s - loss: 4.3557e-04 - mean_squared_error: 4.3557e-04     
Epoch 159/500
537/537 [==============================] - 0s - loss: 4.4613e-04 - mean_squared_error: 4.4613e-04     
Epoch 160/500
537/537 [==============================] - 0s - loss: 4.2947e-04 - mean_squared_error: 4.2947e-04     
Epoch 161/500
537/537 [==============================] - 0s - loss: 3.7374e-04 - mean_squared_error: 3.7374e-04     
Epoch 162/500
537/537 [==============================] - 0s - loss: 4.2090e-04 - mean_squared_error: 4.2090e-04     
Epoch 163/500
537/537 [==============================] - 0s - loss: 5.0309e-04 - mean_squared_error: 5.0309e-04     
Epoch 164/500
537/537 [==============================] - 0s - loss: 3.8564e-04

537/537 [==============================] - 0s - loss: 5.1716e-04 - mean_squared_error: 5.1716e-04     
Epoch 226/500
537/537 [==============================] - 0s - loss: 5.6792e-04 - mean_squared_error: 5.6792e-04     
Epoch 227/500
537/537 [==============================] - 0s - loss: 4.5784e-04 - mean_squared_error: 4.5784e-04     
Epoch 228/500
537/537 [==============================] - 0s - loss: 3.7270e-04 - mean_squared_error: 3.7270e-04     
Epoch 229/500
537/537 [==============================] - 0s - loss: 4.0052e-04 - mean_squared_error: 4.0052e-04     
Epoch 230/500
537/537 [==============================] - 0s - loss: 3.2258e-04 - mean_squared_error: 3.2258e-04     
Epoch 231/500
537/537 [==============================] - 0s - loss: 3.6521e-04 - mean_squared_error: 3.6521e-04     
Epoch 232/500
537/537 [==============================] - 0s - loss: 2.5329e-04 - mean_squared_error: 2.5329e-04     
Epoch 233/500
537/537 [==============================] - 0s - loss: 2.3419e-04

537/537 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012     
Epoch 295/500
537/537 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 296/500
537/537 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 297/500
537/537 [==============================] - 0s - loss: 0.0038 - mean_squared_error: 0.0038     
Epoch 298/500
537/537 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 299/500
537/537 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 300/500
537/537 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010             
Epoch 301/500
537/537 [==============================] - 0s - loss: 7.4402e-04 - mean_squared_error: 7.4402e-04     
Epoch 302/500
537/537 [==============================] - 0s - loss: 7.6680e-04 - mean_squared_error: 7.6680e-04
Epoch 303/500


537/537 [==============================] - 0s - loss: 6.5146e-04 - mean_squared_error: 6.5146e-04     
Epoch 365/500
537/537 [==============================] - 0s - loss: 4.3582e-04 - mean_squared_error: 4.3582e-04     
Epoch 366/500
537/537 [==============================] - 0s - loss: 4.1037e-04 - mean_squared_error: 4.1037e-04     
Epoch 367/500
537/537 [==============================] - 0s - loss: 4.1973e-04 - mean_squared_error: 4.1973e-04     
Epoch 368/500
537/537 [==============================] - 0s - loss: 4.6600e-04 - mean_squared_error: 4.6600e-04     
Epoch 369/500
537/537 [==============================] - 0s - loss: 2.7602e-04 - mean_squared_error: 2.7602e-04     
Epoch 370/500
537/537 [==============================] - 0s - loss: 2.8685e-04 - mean_squared_error: 2.8685e-04     
Epoch 371/500
537/537 [==============================] - 0s - loss: 2.7719e-04 - mean_squared_error: 2.7719e-04     
Epoch 372/500
537/537 [==============================] - 0s - loss: 3.7107e-04

537/537 [==============================] - 0s - loss: 2.2596e-04 - mean_squared_error: 2.2596e-04     
Epoch 435/500
537/537 [==============================] - 0s - loss: 5.7022e-04 - mean_squared_error: 5.7022e-04     
Epoch 436/500
537/537 [==============================] - 0s - loss: 4.8427e-04 - mean_squared_error: 4.8427e-04     
Epoch 437/500
537/537 [==============================] - 0s - loss: 4.5978e-04 - mean_squared_error: 4.5978e-04     
Epoch 438/500
537/537 [==============================] - 0s - loss: 3.4476e-04 - mean_squared_error: 3.4476e-04     
Epoch 439/500
537/537 [==============================] - 0s - loss: 3.6166e-04 - mean_squared_error: 3.6166e-04     
Epoch 440/500
537/537 [==============================] - 0s - loss: 2.9782e-04 - mean_squared_error: 2.9782e-04     
Epoch 441/500
537/537 [==============================] - 0s - loss: 3.1728e-04 - mean_squared_error: 3.1728e-04     
Epoch 442/500
537/537 [==============================] - 0s - loss: 2.4646e-04

### Mean Aggregation

In [12]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


In [13]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [14]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

In [15]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

RMSE:  1.1357523938231158
MAE:  0.7394454356602262
R_Squared:  0.9852211551216941
Adjusted R-squared -  0.9850653833706711
